In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import torch
import os
import glob
import math
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import PIL.Image
import numpy as np

In [3]:
os.chdir('/content/drive/MyDrive/sc2')

In [4]:
!ls

datasetA.zip


In [5]:
!unzip -q datasetA.zip

In [6]:
dataset = datasets.ImageFolder(
    'datasetA',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

dataset_size = len(dataset)
print(dataset_size)

4800


In [7]:
train_ratio = 0.8
val_ratio = 0.2

train_size = math.floor(dataset_size * train_ratio)
val_size = dataset_size - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [8]:

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

In [9]:
print(len(val_loader.dataset))

960


In [10]:
model = models.resnet18(pretrained=True)

# model = models.alexnet(pretrained=True)
# num_classes = 4

# model = models.resnet34(pretrained=True)

# model = models.mobilenet_v2(pretrained=True)

# model = models.vgg16(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]


In [11]:
model.fc = torch.nn.Linear(512, 4)
# resnet18,34

# model.classifier[6] = torch.nn.Linear(4096 ,4)
# alex and vgg16

# model.classifier[1] = torch.nn.Linear(1280 ,4)
# mobibenet

device = torch.device('cuda')
model = model.to(device)

In [12]:
NUM_EPOCHS = 15
BEST_MODEL_PATH = 'best_model_resnet34.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):

    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

    val_error_count = 0.0
    for images, labels in iter(val_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        val_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))

    val_accuracy = 1.0 - float(val_error_count) / float(len(val_dataset))
    print('%d: %f' % (epoch, val_accuracy))
    if val_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = val_accuracy

0: 0.983333
1: 0.998958
2: 1.000000
3: 0.991667
4: 0.996875
5: 1.000000
6: 1.000000
7: 1.000000
8: 1.000000
9: 1.000000
10: 1.000000
11: 1.000000
12: 1.000000
13: 0.998958
14: 1.000000
